In [169]:
import pandas as pd

In [170]:
base_vendas = pd.read_excel('bases/relatorio_notas.xlsx')
display(base_vendas)

,Numero nota fiscal,Data da venda,Código produto,Descrição do produto,Quantidade do produto,Valor unitário,Valor total (nota),Quantidade (volume nota),Valor desconto,Valor total produtos,Unidade
0,564555,06/06/2025,1093,CANETA ESF CR MEDIA 1.0 AZ BIC,100.0,0.620000,0.590950,10.0,0.35,1047.13,UN
1,564555,06/06/2025,3708,PAPEL A4 SENNINHA AZ 75G 100F REPORT,25.0,5.460000,5.132735,10.0,2.69,1047.13,PC
2,564555,06/06/2025,3711,PAPEL A4 SENNINHA VD 75G 100F REPORT,25.0,5.460000,5.133527,10.0,2.69,1047.13,PC
3,564555,06/06/2025,712,REAB MARC PERM AZ 37ML PILOT,12.0,8.120000,7.818223,10.0,4.55,1047.13,UN
4,564555,06/06/2025,12815,QUADRO HORARIO TRABALHO HORIZON DAC 046F,40.0,6.850000,6.195026,10.0,3.84,1047.13,UN
...,...,...,...,...,...,...,...,...,...,...,...
316347,403,03/01/2025,31327,CAIXA TERMICA PT 34L MOR,20.0,71.595537,71.595537,10.0,NaN,NaN,UN
316348,403,03/01/2025,31329,CAIXA TERMICA VM 18L MOR,52.0,44.910000,44.910000,10.0,NaN,NaN,UN
316349,403,03/01/2025,9552,CX ISOPOR 33L ISOPLAST,6.0,25.542042,25.998995,10.0,NaN,NaN,UN
316350,402,03/01/2025,6792,CX ISOPOR 3L ISOPLAST,24.0,2.780000,3.172271,1.0,NaN,NaN,UN


In [171]:
base_vendas.columns.tolist()

['Numero nota fiscal',
 'Data da venda',
 'Código produto',
 'Descrição do produto',
 'Quantidade do produto',
 'Valor unitário',
 'Valor total (nota)',
 'Quantidade (volume nota)',
 'Valor desconto',
 'Valor total produtos',
 'Unidade']

In [172]:
base_vendas.isnull().sum()

Numero nota fiscal             0
Data da venda                  0
Código produto                 0
Descrição do produto           0
Quantidade do produto          0
Valor unitário                 0
Valor total (nota)             0
Quantidade (volume nota)    1700
Valor desconto              2992
Valor total produtos        2710
Unidade                        0
dtype: int64

In [173]:
base_vendas.describe()

,Numero nota fiscal,Código produto,Quantidade do produto,Valor unitário,Valor total (nota),Quantidade (volume nota),Valor desconto,Valor total produtos
count,316352.000000,316352.000000,316352.000000,316352.000000,316352.000000,314652.000000,313360.000000,313642.000000
mean,479163.187396,22072.866016,15.675853,15.841829,13.089965,28.499574,5.588534,2089.542827
std,181381.251400,11739.467571,84.047683,28.998172,23.923867,29.011042,11.322789,3690.735347
min,401.000000,5.000000,-3125.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,544042.000000,11452.000000,2.000000,3.960000,3.184941,11.000000,1.100000,501.510000
50%,550455.000000,24782.000000,5.000000,8.180000,6.649376,19.000000,2.580000,1074.740000
75%,557628.000000,32222.000000,12.000000,18.700000,15.165985,36.000000,6.450000,2334.170000
max,564555.000000,38229.000000,9764.000000,1169.000000,974.762703,238.000000,569.680000,141784.350000


In [174]:
colunas_uteis = [
    'Numero nota fiscal', 
    'Código produto', 
    'Descrição do produto',
    'Quantidade do produto', 
    'Valor unitário', 
    'Valor total produtos',
    'Valor total (nota)'
]

df_vendas = base_vendas[colunas_uteis].copy()

In [175]:
valor_nota = df_vendas.pop('Valor total produtos')
df_vendas['valor_nota'] = valor_nota

In [176]:

df_vendas.rename(columns={'Valor total (nota)': 'Valor pago'}, inplace=True)
df_vendas

,Numero nota fiscal,Código produto,Descrição do produto,Quantidade do produto,Valor unitário,Valor pago,valor_nota
0,564555,1093,CANETA ESF CR MEDIA 1.0 AZ BIC,100.0,0.620000,0.590950,1047.13
1,564555,3708,PAPEL A4 SENNINHA AZ 75G 100F REPORT,25.0,5.460000,5.132735,1047.13
2,564555,3711,PAPEL A4 SENNINHA VD 75G 100F REPORT,25.0,5.460000,5.133527,1047.13
3,564555,712,REAB MARC PERM AZ 37ML PILOT,12.0,8.120000,7.818223,1047.13
4,564555,12815,QUADRO HORARIO TRABALHO HORIZON DAC 046F,40.0,6.850000,6.195026,1047.13
...,...,...,...,...,...,...,...
316347,403,31327,CAIXA TERMICA PT 34L MOR,20.0,71.595537,71.595537,NaN
316348,403,31329,CAIXA TERMICA VM 18L MOR,52.0,44.910000,44.910000,NaN
316349,403,9552,CX ISOPOR 33L ISOPLAST,6.0,25.542042,25.998995,NaN
316350,402,6792,CX ISOPOR 3L ISOPLAST,24.0,2.780000,3.172271,NaN


In [177]:
df_vendas.isnull().sum()

Numero nota fiscal          0
Código produto              0
Descrição do produto        0
Quantidade do produto       0
Valor unitário              0
Valor pago                  0
valor_nota               2710
dtype: int64

In [178]:
print("% dos Valores faltantes")
df_vendas.isnull().sum()/df_vendas['Código produto'].count() * 100

% dos Valores faltantes


Numero nota fiscal       0.000000
Código produto           0.000000
Descrição do produto     0.000000
Quantidade do produto    0.000000
Valor unitário           0.000000
Valor pago               0.000000
valor_nota               0.856641
dtype: float64

In [179]:
df_vendas.dropna(subset=['valor_nota'],axis=0)
df_vendas


,Numero nota fiscal,Código produto,Descrição do produto,Quantidade do produto,Valor unitário,Valor pago,valor_nota
0,564555,1093,CANETA ESF CR MEDIA 1.0 AZ BIC,100.0,0.620000,0.590950,1047.13
1,564555,3708,PAPEL A4 SENNINHA AZ 75G 100F REPORT,25.0,5.460000,5.132735,1047.13
2,564555,3711,PAPEL A4 SENNINHA VD 75G 100F REPORT,25.0,5.460000,5.133527,1047.13
3,564555,712,REAB MARC PERM AZ 37ML PILOT,12.0,8.120000,7.818223,1047.13
4,564555,12815,QUADRO HORARIO TRABALHO HORIZON DAC 046F,40.0,6.850000,6.195026,1047.13
...,...,...,...,...,...,...,...
316347,403,31327,CAIXA TERMICA PT 34L MOR,20.0,71.595537,71.595537,NaN
316348,403,31329,CAIXA TERMICA VM 18L MOR,52.0,44.910000,44.910000,NaN
316349,403,9552,CX ISOPOR 33L ISOPLAST,6.0,25.542042,25.998995,NaN
316350,402,6792,CX ISOPOR 3L ISOPLAST,24.0,2.780000,3.172271,NaN


In [180]:
df_vendas_limpo = df_vendas[df_vendas['valor_nota'].fillna(0) != 0].copy()
df_vendas_limpo

,Numero nota fiscal,Código produto,Descrição do produto,Quantidade do produto,Valor unitário,Valor pago,valor_nota
0,564555,1093,CANETA ESF CR MEDIA 1.0 AZ BIC,100.0,0.620000,0.590950,1047.13
1,564555,3708,PAPEL A4 SENNINHA AZ 75G 100F REPORT,25.0,5.460000,5.132735,1047.13
2,564555,3711,PAPEL A4 SENNINHA VD 75G 100F REPORT,25.0,5.460000,5.133527,1047.13
3,564555,712,REAB MARC PERM AZ 37ML PILOT,12.0,8.120000,7.818223,1047.13
4,564555,12815,QUADRO HORARIO TRABALHO HORIZON DAC 046F,40.0,6.850000,6.195026,1047.13
...,...,...,...,...,...,...,...
315799,16916,8802,AGENDA ANUAL CD COST NAPOLI 176F TILIBRA,4.0,35.770000,21.749064,87.00
315800,16915,21602,CANETA DETEC CEDULA FALSA BLT/1 JOCAR,7.0,4.285714,2.769312,19.39
315801,16914,21602,CANETA DETEC CEDULA FALSA BLT/1 JOCAR,7.0,4.285714,2.769312,19.39
315802,16913,36564,AGENDA ESP ANUAL 336F A5 HELLO DAC,1.0,37.620000,25.529850,25.53


In [181]:
df_vendas_limpo['Quantidade do produto'] = df_vendas_limpo['Quantidade do produto'].astype(int)
df_vendas_limpo['Quantidade do produto'] = df_vendas_limpo['Quantidade do produto'].astype(int)

In [182]:
colunas_floats = ['Valor unitário', 'Valor pago', 'valor_nota']
df_vendas_limpo[colunas_floats] = (df_vendas_limpo[colunas_floats].round(2).astype(float))
df_vendas_limpo

,Numero nota fiscal,Código produto,Descrição do produto,Quantidade do produto,Valor unitário,Valor pago,valor_nota
0,564555,1093,CANETA ESF CR MEDIA 1.0 AZ BIC,100,0.62,0.59,1047.13
1,564555,3708,PAPEL A4 SENNINHA AZ 75G 100F REPORT,25,5.46,5.13,1047.13
2,564555,3711,PAPEL A4 SENNINHA VD 75G 100F REPORT,25,5.46,5.13,1047.13
3,564555,712,REAB MARC PERM AZ 37ML PILOT,12,8.12,7.82,1047.13
4,564555,12815,QUADRO HORARIO TRABALHO HORIZON DAC 046F,40,6.85,6.20,1047.13
...,...,...,...,...,...,...,...
315799,16916,8802,AGENDA ANUAL CD COST NAPOLI 176F TILIBRA,4,35.77,21.75,87.00
315800,16915,21602,CANETA DETEC CEDULA FALSA BLT/1 JOCAR,7,4.29,2.77,19.39
315801,16914,21602,CANETA DETEC CEDULA FALSA BLT/1 JOCAR,7,4.29,2.77,19.39
315802,16913,36564,AGENDA ESP ANUAL 336F A5 HELLO DAC,1,37.62,25.53,25.53


In [183]:
df_vendas_limpo.nunique()

Numero nota fiscal       28293
Código produto            9053
Descrição do produto      8880
Quantidade do produto      411
Valor unitário            9364
Valor pago                8867
valor_nota               24727
dtype: int64

In [184]:
df_vendas_limpo.dtypes

Numero nota fiscal         int64
Código produto             int64
Descrição do produto      object
Quantidade do produto      int64
Valor unitário           float64
Valor pago               float64
valor_nota               float64
dtype: object

In [185]:
df_vendas_limpo.isnull().sum()

Numero nota fiscal       0
Código produto           0
Descrição do produto     0
Quantidade do produto    0
Valor unitário           0
Valor pago               0
valor_nota               0
dtype: int64

In [186]:
# Selecionar apenas as colunas numéricas
coluna_numericas = df_vendas_limpo.select_dtypes(include=['int', 'float']).columns
# contar os valores zerados em cada coluna 
zeros = (df_vendas_limpo[coluna_numericas] == 0).sum()
# Exibir os resultados
print("Valores zerados por colunas")
print(zeros[zeros > 0]) # mostra apenas colunas com valores zero

Valores zerados por colunas
Valor unitário    242
dtype: int64


In [187]:
df_notas_limpo = df_vendas_limpo[df_vendas_limpo["Valor unitário"] != 0].copy()
df_notas_limpo

,Numero nota fiscal,Código produto,Descrição do produto,Quantidade do produto,Valor unitário,Valor pago,valor_nota
0,564555,1093,CANETA ESF CR MEDIA 1.0 AZ BIC,100,0.62,0.59,1047.13
1,564555,3708,PAPEL A4 SENNINHA AZ 75G 100F REPORT,25,5.46,5.13,1047.13
2,564555,3711,PAPEL A4 SENNINHA VD 75G 100F REPORT,25,5.46,5.13,1047.13
3,564555,712,REAB MARC PERM AZ 37ML PILOT,12,8.12,7.82,1047.13
4,564555,12815,QUADRO HORARIO TRABALHO HORIZON DAC 046F,40,6.85,6.20,1047.13
...,...,...,...,...,...,...,...
315799,16916,8802,AGENDA ANUAL CD COST NAPOLI 176F TILIBRA,4,35.77,21.75,87.00
315800,16915,21602,CANETA DETEC CEDULA FALSA BLT/1 JOCAR,7,4.29,2.77,19.39
315801,16914,21602,CANETA DETEC CEDULA FALSA BLT/1 JOCAR,7,4.29,2.77,19.39
315802,16913,36564,AGENDA ESP ANUAL 336F A5 HELLO DAC,1,37.62,25.53,25.53


In [188]:
df_notas_limpo["Diferença real"] = df_notas_limpo["Valor unitário"] - df_notas_limpo["Valor pago"]
df_notas_limpo["Diferença %"] = (df_notas_limpo["Diferença real"] * 100 / df_notas_limpo["Valor unitário"]).round(2)

In [189]:
df_notas_limpo

,Numero nota fiscal,Código produto,Descrição do produto,Quantidade do produto,Valor unitário,Valor pago,valor_nota,Diferença real,Diferença %
0,564555,1093,CANETA ESF CR MEDIA 1.0 AZ BIC,100,0.62,0.59,1047.13,0.03,4.84
1,564555,3708,PAPEL A4 SENNINHA AZ 75G 100F REPORT,25,5.46,5.13,1047.13,0.33,6.04
2,564555,3711,PAPEL A4 SENNINHA VD 75G 100F REPORT,25,5.46,5.13,1047.13,0.33,6.04
3,564555,712,REAB MARC PERM AZ 37ML PILOT,12,8.12,7.82,1047.13,0.30,3.69
4,564555,12815,QUADRO HORARIO TRABALHO HORIZON DAC 046F,40,6.85,6.20,1047.13,0.65,9.49
...,...,...,...,...,...,...,...,...,...
315799,16916,8802,AGENDA ANUAL CD COST NAPOLI 176F TILIBRA,4,35.77,21.75,87.00,14.02,39.19
315800,16915,21602,CANETA DETEC CEDULA FALSA BLT/1 JOCAR,7,4.29,2.77,19.39,1.52,35.43
315801,16914,21602,CANETA DETEC CEDULA FALSA BLT/1 JOCAR,7,4.29,2.77,19.39,1.52,35.43
315802,16913,36564,AGENDA ESP ANUAL 336F A5 HELLO DAC,1,37.62,25.53,25.53,12.09,32.14


In [190]:
df_notas_limpo.describe()

,Numero nota fiscal,Código produto,Quantidade do produto,Valor unitário,Valor pago,valor_nota,Diferença real,Diferença %
count,312427.000000,312427.000000,312427.000000,312427.000000,312427.000000,312427.000000,312427.000000,312427.000000
mean,480308.170597,22068.011257,15.377624,15.898525,13.103218,2096.432972,2.795308,17.941871
std,180118.541958,11746.602365,76.284687,29.004840,23.838700,3695.817141,6.175206,9.564389
min,16912.000000,5.000000,1.000000,0.190000,0.170000,1.040000,-33.300000,-133.970000
25%,544106.000000,11442.000000,2.000000,4.000000,3.210000,505.530000,0.600000,12.150000
50%,550490.000000,24782.000000,5.000000,8.220000,6.690000,1080.520000,1.340000,17.230000
75%,557630.500000,32245.000000,12.000000,18.770000,15.190000,2341.640000,2.980000,22.380000
max,564555.000000,38229.000000,7680.000000,1169.000000,974.760000,141784.350000,373.970000,87.110000


In [193]:
nota_valores = df_notas_limpo[df_notas_limpo["Valor unitário"] < df_notas_limpo["Valor pago"]]
nota_valores


,Numero nota fiscal,Código produto,Descrição do produto,Quantidade do produto,Valor unitário,Valor pago,valor_nota,Diferença real,Diferença %
192,564538,32446,FITA ADESIVA 48X100 TR C/4 ADELBRAS,55,27.29,27.33,3100.20,-0.04,-0.15
247,564529,31440,EVA FOLHA 40X48 VM C/10 BRW,2,10.54,11.09,384.10,-0.55,-5.22
249,564529,31445,EVA FOLHA 40X48 AM CL C/10 BRW PE5007,2,10.54,11.31,384.10,-0.77,-7.31
398,564515,29163,PASTA ELAST 30MM OF CR DELLO,4,3.76,3.83,715.88,-0.07,-1.86
447,564512,31462,EVA FOLHA 40X48 VD AGUA C/10 BRW,1,10.50,10.89,1439.83,-0.39,-3.71
...,...,...,...,...,...,...,...,...,...
315292,17036,34227,ESTOJO ESTAMPADO MAGIC RS 2/ZIP BRW,1,21.28,23.97,93.41,-2.69,-12.64
315404,17015,28203,AGENDA ANUAL COST KAOS 192F SD,4,12.35,12.36,362.72,-0.01,-0.08
315602,16963,13553,EVA FOLHA 40X48 VM C/10 IBEL 61,1,14.28,15.82,868.74,-1.54,-10.78
315607,16963,24866,COLA BR 90G MAXI COLA FRAMA 441,6,1.66,1.69,868.74,-0.03,-1.81


In [194]:
registros_problematicos = df_notas_limpo[
    (df_notas_limpo["Valor unitário"] < df_notas_limpo["Valor pago"]) |
    (df_notas_limpo["Valor unitário"] <= 0) |
    (df_notas_limpo["Valor pago"] <= 0)
]
print(f"Total de registros a serem removidos: {len(registros_problematicos)}")

Total de registros a serem removidos: 4098


In [195]:
df_notas_final = df_notas_limpo[~df_notas_limpo.index.isin(registros_problematicos.index)].copy()

In [196]:
print(f"Total original: {len(df_notas_limpo)}")
print(f"Total após limpeza: {len(df_notas_final)}")
print(f"Registros removidos: {len(df_notas_limpo) - len(df_notas_final)}")

Total original: 312427
Total após limpeza: 308329
Registros removidos: 4098


In [198]:
df_notas_final.isnull().sum()

Numero nota fiscal       0
Código produto           0
Descrição do produto     0
Quantidade do produto    0
Valor unitário           0
Valor pago               0
valor_nota               0
Diferença real           0
Diferença %              0
dtype: int64

In [200]:
df_notas_final.describe()

,Numero nota fiscal,Código produto,Quantidade do produto,Valor unitário,Valor pago,valor_nota,Diferença real,Diferença %
count,308329.000000,308329.000000,308329.000000,308329.000000,308329.000000,308329.000000,308329.000000,308329.000000
mean,480228.033403,22070.052499,14.782388,15.958178,13.117071,2058.761535,2.841108,18.272279
std,180190.365934,11765.488196,70.598132,29.146447,23.937080,3562.960694,6.201224,9.096829
min,16912.000000,5.000000,1.000000,0.190000,0.170000,1.040000,0.000000,0.000000
25%,544091.000000,11431.000000,2.000000,4.010000,3.210000,502.560000,0.620000,12.360000
50%,550472.000000,24782.000000,5.000000,8.230000,6.690000,1070.490000,1.370000,17.360000
75%,557621.000000,32300.000000,12.000000,18.830000,15.180000,2311.320000,3.020000,22.500000
max,564555.000000,38229.000000,7680.000000,1169.000000,974.760000,141784.350000,373.970000,87.110000


In [202]:
base_notas = df_notas_final.to_excel("bases/bases_limpas/base_notas_limpas.xlsx", index=False)